In [11]:
from genetic_algorithm import *
from numpy.random import default_rng
import matplotlib.pyplot as plt


In [2]:

# ## hexadecimal generator

nr_of_input = 3
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
world_size_x = 20
world_size_y = 20

rng = default_rng()
x = rng.choice(world_size_x, size=nr_individuals, replace=False)
y = rng.choice(world_size_y, size=nr_individuals, replace=False)

# ## initial brain and position generator

result = calculate_individual_output_weights(individuals)
print(result)



{0: {'out0': 0.983, 'out3': 0.003, 'out4': 0.003, 'out2': 0.881, 'out1': 0.004}, 1: {'out1': 0.999, 'out0': 0.006}, 2: {'out1': 1.0, 'out2': 0.5, 'out0': 0.02, 'out4': 0.5}, 3: {'out2': 0.551, 'out3': 0.009, 'out1': 0.087}}


In [3]:
for indiv in result:
    result[indiv]['position'] = [(x[indiv], y[indiv])]

In [4]:
result

{0: {'out0': 0.983,
  'out3': 0.003,
  'out4': 0.003,
  'out2': 0.881,
  'out1': 0.004,
  'position': [(1, 3)]},
 1: {'out1': 0.999, 'out0': 0.006, 'position': [(6, 12)]},
 2: {'out1': 1.0,
  'out2': 0.5,
  'out0': 0.02,
  'out4': 0.5,
  'position': [(18, 19)]},
 3: {'out2': 0.551, 'out3': 0.009, 'out1': 0.087, 'position': [(9, 8)]}}

In [5]:
k = 15
n = 0
while k>n:  
    for indiv in result:
        x = result[indiv]['position'][-1][0]
        y = result[indiv]['position'][-1][1]
        position_list = []
        for out in result[indiv]:
            if 'out' in out:
                new_pos = move(x, y, out, result[indiv][out])
                position_list.append(new_pos)
        
        position_list = list(map(sum, zip(*position_list)))
#         print(position_list)
        position_list = make_smaller_(position_list)
#         print(position_list)
        position_list = list(map(sum, zip(*[[x, y]] + [position_list])))

        position_list[0] = normalize_position_if_outside_world(position_list[0], world_size_x)
        position_list[1] = normalize_position_if_outside_world(position_list[1], world_size_y)
        
        
        result[indiv]['position'].append(position_list)
    n += 1
            
result

{0: {'out0': 0.983,
  'out3': 0.003,
  'out4': 0.003,
  'out2': 0.881,
  'out1': 0.004,
  'position': [(1, 3),
   [2, 4],
   [3, 5],
   [4, 6],
   [5, 7],
   [5, 8],
   [6, 9],
   [7, 10],
   [8, 11],
   [9, 12],
   [10, 13],
   [11, 14],
   [12, 15],
   [12, 16],
   [13, 17],
   [14, 18]]},
 1: {'out1': 0.999,
  'out0': 0.006,
  'position': [(6, 12),
   [6, 11],
   [6, 10],
   [6, 9],
   [6, 8],
   [6, 7],
   [6, 6],
   [6, 5],
   [6, 4],
   [6, 3],
   [6, 2],
   [6, 1],
   [6, 0],
   [6, 0],
   [6, 0],
   [6, 0]]},
 2: {'out1': 1.0,
  'out2': 0.5,
  'out0': 0.02,
  'out4': 0.5,
  'position': [(18, 19),
   [18, 18],
   [19, 17],
   [18, 16],
   [18, 15],
   [17, 14],
   [17, 13],
   [17, 12],
   [17, 11],
   [17, 10],
   [17, 9],
   [16, 8],
   [16, 7],
   [16, 6],
   [17, 5],
   [17, 4]]},
 3: {'out2': 0.551,
  'out3': 0.009,
  'out1': 0.087,
  'position': [(9, 8),
   [9, 8],
   [9, 8],
   [9, 8],
   [10, 8],
   [11, 8],
   [12, 8],
   [13, 8],
   [14, 8],
   [15, 8],
   [16, 8],
   

In [22]:
[9, 8]==[9, 8]

True

In [20]:
slope_intercept(9, 8, 9, 8)

ZeroDivisionError: division by zero

In [7]:
a = [(3, 0),(4,1),(5,2)]

def slope_intercept(x1,y1,x2,y2):
    a = (y2 - y1) / (x2 - x1)
    b = y1 - a * x1     
    return a,b

# print(slope_intercept(3,0,2,1))

if a[-2][0] == a[-1][0]:
    x = a[-1][0]
elif a[-2][0] > a[-1][0]:
    x = a[-2][0] + 1
elif a[-2][0] < a[-1][0]:
    x = a[-2][0] - 1
# print(x)

a[-2][0]



4